In [1]:
import numpy as np 
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 

from SmithZero import D2torchEngine

In [2]:
torch.manual_seed(42)

In [3]:
from torchvision.datasets import CIFAR10 
import torchvision.transforms as T


# === data transformation === # 
train_T = T.Compose([   T.RandomCrop(32, padding=4),
                        T.RandomHorizontalFlip(), 
                        T.ToTensor(), 
                        T.Normalize(mean= (0.4914, 0.4822, 0.4465),
                                    std=(0.2023, 0.1994, 0.2010)),
                    ])

test_T = T.Compose([T.ToTensor(),
                    T.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                std=(0.2023, 0.1994, 0.2010))
                    ])                  


# === download dataset object === # 
train_data = CIFAR10 (  root="./dataset/train",
                        train=True,
                        download=True,
                        transform=train_T )

test_data = CIFAR10 (   root="./dataset/test",
                        train=False,
                        download=True, 
                        transform=test_T )  

Files already downloaded and verified
Files already downloaded and verified


In [4]:
from torch.utils.data import DataLoader

batchSize = 124

testloader = DataLoader(test_data,
                        batch_size=batchSize,
                        num_workers=4 
                        )

In [5]:
class LeNet(nn.Module):
    def __init__(self, in_channels=3, n_class=10, p=0.5):
        super(LeNet, self).__init__()
        self.p = p # probability for DropOut layer 
        
        # === Create the convolution layers === # 
        self.c1 = nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=5, padding=2)
        self.c3 = nn.Conv2d(6, 16, 5) 
        self.c5 = nn.Conv2d(16, 120, 5) 

        # === Create the linear layers === # 
        self.f6 = nn.Linear(in_features=480, out_features=84)
        self.output = nn.Linear(in_features=84, out_features=n_class)

        # === Create dropout layers === # 
        self.drop = nn.Dropout(self.p)

    def featurizer(self, x):
        # === block1 === # 
        x = self.c1(x)
        x = F.relu(x) 
        x = F.max_pool2d(x, kernel_size=2)
        # === block2 === # 
        x = self.c3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        # === block3 === # 
        x = self.c5(x)
        x = F.relu(x)
        # ==== flattening === #
        x = nn.Flatten()(x)
        return x 

    def classifier(self, x): 
        # === hidden layler === # 
        if self.p > 0: 
            x = self.drop(x)        
        x = self.f6(x)
        x = F.relu(x)

        # === output layer === # 
        if self.p > 0 :
            x = self.drop(x)
        x = self.output(x)
        return x 
    
    def forward(self, x):
        x = self.featurizer(x) # return (1, 480)
        x = self.classifier(x) # return (1, 10)
        return x 

In [6]:
model = LeNet(in_channels=3, n_class=10, p=0.5)

loss_fn = nn.CrossEntropyLoss() 
optimizer = optim.SGD( model.parameters(), # (!) be sure to pass in the model.parameters() 
                        lr=1e-3, 
                        momentum=0.9,
                    ) 

In [7]:
AgentDL = D2torchEngine(model, loss_fn, optimizer)  # init. your deep learning engine 

/home/milky/anaconda3/envs/py38/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
AgentDL.load_checkpoint('model_checkpoint.pth')

***
### Multiclass Classifcation Accuracy

In [9]:
inputs, labels = next(iter(testloader))

In [10]:
results = AgentDL.correct(inputs, labels)

for cls_idx, (num_correct, num_items) in enumerate(results.tolist()):
    print(f"cls_idx_{cls_idx}: {num_correct}/{num_items}")


sum_correct = results[:,0].sum().item()
total_items = results[:,1].sum().item()

print(f"Accuracy: {sum_correct/total_items:.3f}")

cls_idx_0: 7/12
cls_idx_1: 9/10
cls_idx_2: 5/11
cls_idx_3: 7/14
cls_idx_4: 5/10
cls_idx_5: 2/9
cls_idx_6: 15/19
cls_idx_7: 8/13
cls_idx_8: 15/15
cls_idx_9: 10/11
Accuracy: 0.669


In [11]:
results = [AgentDL.correct(inputs, labels) for idx, (inputs, labels) in enumerate(testloader)]

print(len(results))
print(len(test_data)/ batchSize)

81
80.64516129032258


In [12]:
results = torch.stack(results, axis=0)
print(results.size())

torch.Size([81, 10, 2])


In [13]:
# Sum accuracy 
assemble = results.sum(axis=0)

for idx, (n_correct, n_items) in enumerate(assemble.tolist()):
    print(f"cls_idx_{idx}: {n_correct}/{n_items}")

sum_correct = assemble[:,0].sum().item()
total_items = assemble[:,1].sum().item()

print(f"Accuracy: {sum_correct/total_items:.3f}")

cls_idx_0: 743/1000
cls_idx_1: 921/1000
cls_idx_2: 514/1000
cls_idx_3: 457/1000
cls_idx_4: 685/1000
cls_idx_5: 563/1000
cls_idx_6: 724/1000
cls_idx_7: 764/1000
cls_idx_8: 767/1000
cls_idx_9: 774/1000
Accuracy: 0.691


In [14]:
# Mean accuracy 
assemble = results.float().mean(axis=0)

for idx, (n_correct, n_items) in enumerate(assemble.tolist()):
    print(f"cls_idx_{idx}: {n_correct:.3f}/{n_items:.3f}")

sum_correct = assemble[:,0].sum().item()
total_items = assemble[:,1].sum().item()

print(f"Accuracy: {sum_correct/total_items:.3f}")

cls_idx_0: 9.173/12.346
cls_idx_1: 11.370/12.346
cls_idx_2: 6.346/12.346
cls_idx_3: 5.642/12.346
cls_idx_4: 8.457/12.346
cls_idx_5: 6.951/12.346
cls_idx_6: 8.938/12.346
cls_idx_7: 9.432/12.346
cls_idx_8: 9.469/12.346
cls_idx_9: 9.556/12.346
Accuracy: 0.691


### Classy - ([ref](https://github.com/dvgodoy/PyTorchStepByStep/blob/master/Chapter05.ipynb))
* multiclass classfication accuracy

In [15]:
# add attribute test 

class TEST(object):
    def __init__(self):
        self.x = 10 

# ----------
def add_method():
    print("hi")


test_model = TEST()
setattr(test_model,'add_method', add_method)

test_model.add_method()

hi


In [16]:
@staticmethod 
def loader_apply(dataloader, func, reduce='sum'):
    results = [func(inputs, labels) for idx, (inputs, labels) in enumerate(dataloader)]
    results = torch.stack(results, axis=0)

    if reduce == 'sum': 
        results = results.sum(axis=0)
    elif reduce == 'mean': 
        results = results.float().mean(axis=0)

    return results 

setattr(D2torchEngine, 'loader_apply', loader_apply) # add method attribute

In [17]:
AgentDL.set_loaders(train_loader=None, val_loader=testloader)

results = D2torchEngine.loader_apply(AgentDL.val_loader, AgentDL.correct, reduce='sum')
print(results)


for idx, (n_correct, n_items) in enumerate(results.tolist()):
    print(f"cls_idx_{idx}: {n_correct}/{n_items}")


results = results.sum(axis=0)
print(results)
print(f"Accuracy: {results[0]/results[1]:.3f}")

tensor([[ 743, 1000],
        [ 921, 1000],
        [ 514, 1000],
        [ 457, 1000],
        [ 685, 1000],
        [ 563, 1000],
        [ 724, 1000],
        [ 764, 1000],
        [ 767, 1000],
        [ 774, 1000]])
cls_idx_0: 743/1000
cls_idx_1: 921/1000
cls_idx_2: 514/1000
cls_idx_3: 457/1000
cls_idx_4: 685/1000
cls_idx_5: 563/1000
cls_idx_6: 724/1000
cls_idx_7: 764/1000
cls_idx_8: 767/1000
cls_idx_9: 774/1000
tensor([ 6912, 10000])
Accuracy: 0.691


In [18]:
AgentDL.set_loaders(train_loader=None, val_loader=testloader)

results = D2torchEngine.loader_apply(AgentDL.val_loader, AgentDL.correct, reduce='mean')
print(results)


for idx, (n_correct, n_items) in enumerate(results.tolist()):
    print(f"cls_idx_{idx}: {n_correct:.3f}/{n_items:.3f}")


results = results.sum(axis=0)
print(results)
print(f"Accuracy: {results[0]/results[1]:.3f}")

tensor([[ 9.1728, 12.3457],
        [11.3704, 12.3457],
        [ 6.3457, 12.3457],
        [ 5.6420, 12.3457],
        [ 8.4568, 12.3457],
        [ 6.9506, 12.3457],
        [ 8.9383, 12.3457],
        [ 9.4321, 12.3457],
        [ 9.4691, 12.3457],
        [ 9.5556, 12.3457]])
cls_idx_0: 9.173/12.346
cls_idx_1: 11.370/12.346
cls_idx_2: 6.346/12.346
cls_idx_3: 5.642/12.346
cls_idx_4: 8.457/12.346
cls_idx_5: 6.951/12.346
cls_idx_6: 8.938/12.346
cls_idx_7: 9.432/12.346
cls_idx_8: 9.469/12.346
cls_idx_9: 9.556/12.346
tensor([ 85.3333, 123.4568])
Accuracy: 0.691
